## Apresentação de dados usando Bokeh 
#### Biblioteca interativa de visualização para em browsers web que provê uma construção concisa, elegante e versátil de gráficos, auxiliando na contrução rápida e fácil de dashboards.

In [1]:
#!pip install bokeh

### Importação / Carga dos dados 
(Obs a aplicação do governo muda o  endereço de acesso, por isso os dados são referentes ao dia 10/04/2020)

In [2]:
import pandas as pd
covid_br = pd.read_csv(
    'https://mobileapps.saude.gov.br/esus-vepi/files/unAFkcaNDeXajurGB7LChj8SgQYS2ptm/4101ebf78c503bf35ecba5545d236c76_Download_COVID19_20200410.csv',
    ';',
    encoding='iso8859')
covid_br.head()

,region,sigla,date,cases_inc,cases,deaths_inc,deaths
0,Norte,RO,30/01/2020,0,0,0,0
1,Norte,RO,31/01/2020,0,0,0,0
2,Norte,RO,01/02/2020,0,0,0,0
3,Norte,RO,02/02/2020,0,0,0,0
4,Norte,RO,03/02/2020,0,0,0,0


### Tranformação dos dados para formato de data adequado e ordenação com base na data

In [3]:
covid_br['nova_data'] = pd.to_datetime(covid_br['date'],format="%d/%m/%Y")
covid_br.sort_values('nova_data').head()

,region,sigla,date,cases_inc,cases,deaths_inc,deaths,nova_data
0,Norte,RO,30/01/2020,0,0,0,0,2020-01-30
1872,Centro-Oeste,DF,30/01/2020,0,0,0,0,2020-01-30
72,Norte,AC,30/01/2020,0,0,0,0,2020-01-30
864,Nordeste,PE,30/01/2020,0,0,0,0,2020-01-30
1080,Nordeste,BA,30/01/2020,0,0,0,0,2020-01-30


### Filtrando informações relativas ao Espirito Santo e São Paulo

In [4]:
ESSP = (covid_br[(covid_br['sigla'] == 'ES') | (covid_br['sigla'] == 'SP')]
.loc[:, ['sigla','nova_data', 'date','cases_inc','cases','deaths_inc','deaths']]
.sort_values(['sigla','nova_data']))
ESSP.head()

,sigla,nova_data,date,cases_inc,cases,deaths_inc,deaths
1224,ES,2020-01-30,30/01/2020,0,0,0,0
1225,ES,2020-01-31,31/01/2020,0,0,0,0
1226,ES,2020-02-01,01/02/2020,0,0,0,0
1227,ES,2020-02-02,02/02/2020,0,0,0,0
1228,ES,2020-02-03,03/02/2020,0,0,0,0


### Criangro gráfico e apresentando resultados

In [5]:
import numpy as np

# Bokeh libraries
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.plotting import reset_output
# Use reset_output() between subsequent show() calls, as needed
reset_output()

# Isolate the data for ES and SP
sp_data = ESSP[ESSP['sigla'] == 'SP']
es_data = ESSP[ESSP['sigla'] == 'ES']

# My word count data
total_registros = len(np.array(sp_data.cases))
day_num = np.linspace(1, total_registros, total_registros)
covid_novos_es = np.array(es_data.cases_inc)
es_cumulative = np.array(es_data.cases)
covid_novos_sp = np.array(sp_data.cases_inc)
sp_cumulative = np.array(sp_data.cases)

# Output the visualization directly in the notebook
output_notebook()

# Create a figure with a datetime type x-axis
fig = figure(title='My Tutorial Progress',
             plot_height=500, plot_width=900,
             x_axis_label='Day Number', y_axis_label='Words Written',
             x_minor_ticks=2, y_range=(0, (sp_cumulative.max())),
             toolbar_location=None)


# The daily words will be represented as vertical bars (columns)
fig.vbar(x=day_num, bottom=0, top=covid_novos_sp, 
         color='gray', width=0.85, 
         legend_label='Daily')


# The daily words will be represented as vertical bars (columns)
fig.vbar(x=day_num, bottom=0, top=covid_novos_es, 
         color='blue', width=0.85, 
         legend_label='DailySP')

# The cumulative sum will be a trend line
fig.line(x=day_num, y=sp_cumulative, 
         color='gray', line_width=1,
         legend_label='Cumulative')

# The cumulative sum will be a trend line
fig.line(x=day_num, y=es_cumulative, 
         color='red', line_width=1,
         legend_label='CumulativeSP')


# Put the legend in the upper left corner
fig.legend.location = 'top_left'

# Let's check it out
show(fig)

Loading BokehJS ...

In [6]:
# Bokeh libraries
import pandas as pd
from bokeh.plotting import figure, show,reset_output
from bokeh.io import output_file,output_notebook
from bokeh.models import ColumnDataSource

# Use reset_output() between subsequent show() calls, as needed
reset_output()

# notebook output
output_notebook()

# Output to file
#output_file('corona_essp.html',      title='Corona Virus - Comparativo SP/ES')

# Isolate the data for ES and SP
sp_data = ESSP[ESSP['sigla'] == 'SP']
es_data = ESSP[ESSP['sigla'] == 'ES']

# Create a ColumnDataSource object for each state
sp_cds = ColumnDataSource(sp_data)
es_cds = ColumnDataSource(es_data)

# Create and configure the figure
fig = figure(
             plot_height=500, plot_width=900,
             x_axis_type="datetime",
             title='Corona Virus - Comparativo SP/ES',
             x_axis_label='Date', y_axis_label='Cases',
             toolbar_location='right',
             y_range=(0, ESSP.cases.max()+10),
             x_range=(ESSP.nova_data.min(), ESSP.nova_data.max()))

fig.line('nova_data', 'cases', 
         color='#CE1141', legend_label='SP', 
         source=sp_cds)

fig.vbar(x=sp_data.nova_data, bottom=0, top=sp_data.cases, 
         color='red', width=0.75, 
         legend_label='Diario SP')

fig.line('nova_data', 'cases', 
         color='#006BB6', legend_label='ES', 
         source=es_cds)

fig.vbar(x=es_data.nova_data, bottom=0, top=es_data.cases, 
         color='blue', width=0.75, 
         legend_label='Diario ES')

# Move the legend to the upper left corner
fig.legend.location = 'top_left'

# Show the plot
show(fig)

Loading BokehJS ...

### References
https://docs.python.org/3/library/codecs.html#standard-encodings<br>
https://docs.python.org/3/library/datetime.html#datetime.date.day<br>
https://docs.bokeh.org/en/latest/docs/user_guide/interaction/legends.html<br>
https://docs.bokeh.org/en/latest/docs/user_guide/plotting.html
https://hub.gke.mybinder.org/user/bokeh-bokeh-notebooks-y0pylq7v/notebooks/tutorial/00%20-%20Introduction%20and%20Setup.ipynb